# Mean variance analysis

## 1. Install package and load data

Install the package 'fPortfolio' that we will use extensively for portfolio construction and backtesting. The documentation can be found here: 
https://www.rdocumentation.org/packages/fPortfolio/versions/3042.83.1.
You can also download the following book where it explains how to use the package, including a lot of relevant information regarding models and applications:
https://www.rmetrics.org/ebooks-portfolio.

In [7]:

install.packages('fPortfolio')
library('fPortfolio')


NameError: name 'install' is not defined

Load a test data set to perform some analysis. The data contains the time series of the following monthly returns in percentages:
- Mkt: Market return
- Rf: Risk-free return
- Bnds10: Return on 10-year treasure bonds
- Cnsmr: Return on consumer goods industry portfolio
- Manuf: Return on manufacturing goods industry portfolio
- HiTec: Return on high-tech industry portfolio
- Hlth: Return on health industry portfolio
- Other: Return on other industry portfolio
- SMB: Return on SMB factor portfolio
- HML: Return on HML factor portfolio
- MOM: Return on momentum factor portfolio

The first column contains the date of the returns.

In [8]:
rm(list=ls())              #clear the workspace
# data <- readSeries(file.choose(), header = TRUE, sep = ",", format = "%Y-%m-%d")
all_data <- read("test_data.csv", header = TRUE, sep = ",", format = "%Y-%m-%d")

ERROR: Error in read("test_data.csv", header = TRUE, sep = ",", format = "%Y-%m-%d"): non trovo la funzione "read"


 We will consider portfolios of the five industries and the bonds, with or without a risk-free rate. So let us choose columns 3 to 8 from the dataset.

In [3]:
data_ret <- all_data[,3:8];
data_rf <- all_data[,2];                           #risk-free rate data
data_eret <- sweep(data_ret, 1, data_rf, "-")      #compute excess returns

ERROR: Error in eval(expr, envir, enclos): oggetto "all_data" non trovato


## 2. Portfolio analysis

Let us start with analyzing the properties of an equally weighted portfolio. We specify the portfolio and then set all weights to 1/n where n is the number of assets (nAssets).

In [ ]:
ewSpec <- portfolioSpec()
nAssets <- ncol(data_ret)                              #number of assets
aNames <- colnames(data_ret)                           #store the asset names
setWeights(ewSpec) <- rep(1/nAssets, times = nAssets)
print(ewSpec)

We will now compute the properties of the portfolio, i.e., the mean, standard deviation, CVaR and VaR risk measures, and other statistics.

In [ ]:
ewPortfolio <- feasiblePortfolio(data = data_ret, spec = ewSpec)
print(ewPortfolio)

Let us compute the raw Sharpe ratio of the portfolio by getting the mean and standard deviation of the portfolio.

In [ ]:
mean_ew <- getTargetReturn(ewPortfolio)["mean"]
std_ew <- getTargetRisk(ewPortfolio)["Sigma"]
rSR_ew <- mean_ew[1] / std_ew[1]
names(rSR_ew) <- 'raw SR'                       #change the name of the numbe
rSR_ew

## 3. Tangent portfolio

### Unconstrained

Now we will compute the unconstrained tangent portfolio, i.e. the efficient portfolio of risky assets with the highest Sharpe ratio where we are allowed to take unlimited short positions. Since this problem can be solved in closed form, we need to change the solver. 

The most appropriate approach is to compute the mean vector and the covariance matrix of excess returns. This is because computing the covariance matrix with the raw returns it also includes the covariation that arises from the variation in the risk-free rate. So we want to remove it. Therefore we use the excess returns (data_eret) instead of the raw returns (data_ret). 

Note that we do not need to specify the risk-free rate whose default value is zero. The risk-free rate only affects the mean return of the portfolio. To specify the risk-free rate other than zero, e.g. 0.1, we need to use the command 'setRiskFreeRate(portSpec) <- 0.1'.

In [ ]:
shortSpec <- portfolioSpec()
setSolver(shortSpec) <- "solveRshortExact"
unctgPortfolio <- tangencyPortfolio(data = data_eret, spec = shortSpec)
print(getWeights(unctgPortfolio))

We will now compute the portfolio ourselves from the mean vector and the covariance matrix to see that we obtain the same result.

In [ ]:
Sigma <- cov(data_eret)       #covariance matrix of excess returns
Mu <- colMeans(data_eret)     #mean vector of excess returns
w <- c(inv(Sigma) %*% Mu)     #call the operation inside c() to make it a vector
w <- w / sum(w)               #normalize for the weights to sum to one
names(w) <- aNames
print(w)

We also compute the mean excess returns, the standard deviation of excess returns, and the Sharpe ratio of the tangent portfolio.

In [ ]:
port_stats <- c(w %*% Mu, sqrt(w %*% Sigma %*% w), 0)
port_stats[3] <- port_stats[1] / port_stats[2]
names(port_stats) <- c('Avg(e-ret)', 'Std(e-ret)', 'Sharpe ratio')
print(port_stats)

### Portfolio constraints

We can now compute the tangent portfolio with portfolio constrints. Let us start with no short-selling constraints, i.e., the weights can be anywhere between 0 (lower bound) and 1 (upper bound).

In [ ]:
noshortSpec <- portfolioSpec()
noshorttgPort <- tangencyPortfolio(data = data_eret, spec = noshortSpec, constraints = "LongOnly")
print(getWeights(noshorttgPort))
port_stats <- c(getTargetReturn(noshorttgPort)['mean'], getTargetRisk(noshorttgPort)['Sigma'], 0)
port_stats[3] <- port_stats[1] / port_stats[2]
names(port_stats) <- c('Avg(e-ret)', 'Std(e-ret)', 'Sharpe ratio')
print(port_stats)

Now we will specify portfolio constraints for each asset separaterly. The default values are 0 for lower bounds and 1 for upper bounds. If constraints are not set the bounds are the default ones.

In [ ]:
port_minW <- "minW[1:6] = 0.1"                    #set all minimum weights to 0.1
port_maxW <- "maxW[c(1,5,6)] = c(0.5, 0.6, 0.7)"  #set max weights of assets 1, 5, and 6 to 0.5, 0.6, and 0.7, respectively
portConstraints <- c(port_minW, port_maxW)
portSpec <- portfolioSpec()
tgPort <- tangencyPortfolio(data = data_eret, spec = portSpec, constraints = portConstraints)
print(getWeights(tgPort))

## 4. Target mean or volatility

If we want to find the efficient mean-variance portfolio for a target mean and volatility, the procedure depends on whether we have a risk-free rate or not. If there is a risk-free rate then the first step is to obtain the tangent portfolio. The second step is to scale up (leverage) or down the tangent portfolio to obtain either the target mean
$$ l = \frac{\mu_{target} - r_f}{\mu_{tangent} - r_f}$$
or the target volatility
$$ l = \frac{\sigma_{target}}{\sigma_{tangent}}$$
The portfolio weights will then be $1-l$ in the risk-free rate and $l$ in the tangent portfolio.

However, in the case where we do not have a risk-free rate, then we need to use the function 'efficientPortfolio'. In the example below we will assume no short-selling constraint. The target return should be the target excess return if we are providing excess returns, while the risk-free rate should be left at zero.

In [ ]:
portSpec <- portfolioSpec()
setTargetReturn(portSpec) <- 0.5
targetPort <- efficientPortfolio(data = data_eret, spec = portSpec, constraints = "LongOnly")
w <- getWeights(targetPort)
print(w)

port_stats <- c(getTargetReturn(targetPort)['mean'], getTargetRisk(targetPort)['Sigma'], 0)
port_stats[3] <- port_stats[1] / port_stats[2]
names(port_stats) <- c('Avg(e-ret)', 'Std(e-ret)', 'Sharpe ratio')
print(port_stats)

The optimization to find a portfolio for a target risk does not seem to work. Instead we can search using the mean targeted portfolios.

In [ ]:
targetvol <- 2.5              #target risk
targetmean <- 0.5             #initial target mean

#compute error of intial target mean
portSpec <- portfolioSpec()
setTargetReturn(portSpec) <- targetmean
targetPort <- efficientPortfolio(data = data_eret, spec = portSpec, constraints = "LongOnly")
error <- targetvol - getTargetRisk(targetPort)['Sigma']

#loop until it gets close enough to the target risk
while (abs(error) > 0.001) {
    #update targetmean in the direction of the error
    targetmean <- targetmean + 0.1*error
    
    #compute new error
    setTargetReturn(portSpec) <- targetmean
    targetPort <- efficientPortfolio(data = data_eret, spec = portSpec, 
                                     constraints = "LongOnly")
    error <- targetvol - getTargetRisk(targetPort)['Sigma']
}

w <- getWeights(targetPort)
print(w)
port_stats <- c(getTargetReturn(targetPort)['mean'], getTargetRisk(targetPort)['Sigma'], 0)
port_stats[3] <- port_stats[1] / port_stats[2]
names(port_stats) <- c('Avg(e-ret)', 'Std(e-ret)', 'Sharpe ratio')
print(port_stats)

## Frontiers
Now let us plot the efficient frontiers with no short-selling constraints. Other frontiers can be plotted in a similar way. Below we use the function 'tailoredFrontierPlot' that adds several plots.

In [ ]:
#noshortSpec created earlier
noshortFront <- portfolioFrontier(data = data_eret, spec = noshortSpec, constraints = "LongOnly")
tailoredFrontierPlot(noshortFront, risk = "Sigma", sharpeRatio = FALSE, title = TRUE)

There are also functions to add frontiers one by one. Let us plot the constrained (no short selling) and unconstrained efficient frontiers side by side, with and without risk-free rate.

In [ ]:
frontierPlot(noshortFront, risk = "Sigma", title = TRUE)

#frontierPlot(object, frontier = c("both", "lower", "upper"),
#    col = c("black", "grey"), add = FALSE, labels = TRUE,
#    return = c("mean", "mu"), risk = c("Cov", "Sigma", "CVaR", "VaR"),
#    auto = TRUE, title = TRUE, …) 

#shortSpec created earlier
shortFront <- portfolioFrontier(data = data_eret, spec = shortSpec)
frontierPlot(shortFront, risk = "Sigma", title = TRUE, add = TRUE)

#add tangent lines and assets
tangencyLines(shortFront, risk = "Sigma", col = "blue")
tangencyLines(noshortFront, risk = "Sigma", col = "darkgreen")
singleAssetPoints(shortFront, risk = "Sigma", col = "red")
